In [1]:
### Book to collect data about the current college hockey season from College Hockey News

## Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import logging
from tqdm import tqdm
import re

from sqlalchemy import create_engine


import sqlite3


FILE_TAG = '2001_Season_v1'

## global variables
# # current_year_url = 'https://www.collegehockeynews.com/schedules/?season=20232024'
# tag = '2023_Season_Nov 2'

##### TEMP####
## global variables
current_year_url = 'https://www.collegehockeynews.com/schedules/?season=20212022'
tag = '2022_Season_Nov 2_with_update_notebook'


## Base usl for box scores and metrics
base_url = 'https://www.collegehockeynews.com'



In [2]:
## Functions
### Parse the current season schedule / results page

def parse_current_season(url):
        # Initialize variables
    current_date = None
    current_conference = None
    game_notes = None

    # Initialize an empty list to hold the data
    data = []

    # Parse the page with BeautifulSoup
    # Get the page with requests
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # select the table or tables
    tables = soup.find_all('table')

    rows = soup.find_all('tr')

    # Loop through each row to find relevant information
    for row in rows:
        # Check for date row
        if row.get('class') == ['stats-section']:
            current_date = row.find('td').text.strip()
        # Check for conference row
        elif row.get('class') == ['sked-header']:
            current_conference = row.find('td').text.strip()
        # Check for game notes
        elif len(row.find_all('td')) == 2:
            game_notes = row.find_all('td')[1].text.strip()
        # Process rows with game data
        elif row.get('valign') == 'top':
            cells = row.find_all('td')
            if len(cells) >= 9:
                home_team = cells[0].text.strip()
                home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
                home_score = cells[1].text.strip()
                away_team = cells[3].text.strip()
                away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
                away_score = cells[4].text.strip()
                ot = cells[5].text.strip()
                box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
                metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
                # Capture Game Notes
                game_notes_cell = cells[-1].find('small')
                game_notes = game_notes_cell.text.strip() if game_notes_cell else None

                # Append data to the list
                data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
                game_notes = None  # Reset game notes for the next row
    return data

## call the function
data = parse_current_season(current_year_url)


# Create a dataframe from the list

columns = ['Date', 'Conference', 'Game_Notes', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)
            
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)

## Filter out games that have not been played yet
df = df[df['Home_Score'] != '']

# Replace Nan values in metrics column with empty string
df['Metrics_Link'] = df['Metrics_Link'].fillna('')



C:\Users\Justin\AppData\Local\Temp\ipykernel_13308\2541037650.py:83: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2021-10-02_Army_Providence' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row.Index, 'Game_ID'] = game_id


In [3]:
print(len(df))
df.tail(10)

## Save csv of just the current season results
# df.to_csv('../TEMP/current_season.csv', index=False)

# Store the dataframe as games_df
games_df = df.copy()

1106


In [4]:
## Functions for parsing the box score and metrics pages

# Initialize logging for Error and Warning messages
logging.basicConfig(filename='../TEMP/current_scrape.log', level=logging.INFO)

#### PARSE PLAYER STATS TABLE ####
def parse_player_summary(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the playersums div
    playersums_div = soup.find('div', id='playersums')
    if playersums_div is None:
        return "Player summaries div not found"

    # Initialize list to store player stats
    player_stats = []

    # Loop through each playersum div
    for player_sum in playersums_div.find_all('div', class_='playersum'):
        team = player_sum.find('td').text.strip()
        
        # Loop through table rows
        for row in player_sum.find_all('tr'):
            cols = row.find_all('td')
            if len(cols) > 1:
                player = cols[0].text.strip()
                goals = cols[1].text.strip()
                assists = cols[2].text.strip()
                points = cols[3].text.strip()
                plus_minus = cols[4].text.strip()
                shots = cols[5].text.strip()
                pim = cols[6].text.strip()
                fowl = cols[7].text.strip() if len(cols) > 7 else None
                
                fow, fol = None, None
                win_percentage = None
                
                

                try:
                    if fowl and '‑' in fowl:  # Checking if it contains a hyphen
                        fow, fol = map(int, fowl.split('‑'))
                        total_fo = fow + fol
                        win_percentage = (fow / total_fo) * 100 if total_fo > 0 else 0
                except ValueError:
                    fow, fol, win_percentage = None, None, None

                

                
                player_stat = {
                    'Team': team,
                    'Player': player,
                    'G': goals,
                    'A': assists,
                    'Pt.': points,
                    '+/-': plus_minus,
                    'Sh': shots,
                    'PIM': pim,
                    'FOW': fow,
                    'FOL': fol,
                    'FO%': win_percentage
                }
                player_stats.append(player_stat)

    return pd.DataFrame(player_stats)


############# PARSEING SCORING SUMMARY WITH BS4

def parse_scoring_summary(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the scoring div and table
    scoring_div = soup.find('div', id='scoring')
    if scoring_div is None:
        logging.error("Scoring div not found")
        return None

    scoring_table = scoring_div.find('table')
    if scoring_table is None:
        logging.error("Scoring table not found within the scoring div")
        return None

    # Initialize list to store scoring events
    scoring_events = []
    period = None

    # Loop through table rows
    for row in scoring_table.find_all('tr'):
        if 'stats-section' in row.get('class', []):
            td = row.find('td')
            if td:
                period = td.text.strip()
            else:
                logging.warning("Period name not found in 'stats-section' row")
                period = "Unknown"
        else:
            cols = row.find_all('td')
            if len(cols) > 1:
                try:
                    team = cols[0].text.strip()
                    pp = cols[1].text.strip()

                    player_data = cols[3].text.strip()
                    match = re.match(r"(.+)\s\((\d+)\)", player_data)
                    player = match.group(1) if match else player_data
                    goals = int(match.group(2)) if match else None

                    assist_data_raw = cols[4].text.strip()
                    assist_data = assist_data_raw.split(", ") if assist_data_raw else []
                    assist1 = assist_data[0] if len(assist_data) > 0 else None
                    assist2 = assist_data[1] if len(assist_data) > 1 else None

                    time = cols[5].text.strip()

                    scoring_event = {
                        'Period': period,
                        'Team': team,
                        'PP': pp,
                        'Player': player,
                        'Player_Goals': goals,
                        'Assist1': assist1,
                        'Assist2': assist2,
                        'Time': time
                    }
                    scoring_events.append(scoring_event)
                except Exception as e:
                    logging.error(f"An error occurred while parsing a scoring event row: {e}")
            else:
                logging.warning(f"Insufficient columns in scoring row: {len(cols)}")

    return pd.DataFrame(scoring_events)


############# PARSEING PENALTY SUMMARY WITH BS4

def parse_penalty_summary(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the penalties div and table
    penalties_div = soup.find('div', id='penalties')
    if penalties_div is None:
        logging.error("Penalties div not found")
        return None

    penalties_table = penalties_div.find('table')
    if penalties_table is None:
        logging.error("Penalties table not found within the penalties div")
        return None

    # Initialize list to store penalty events
    penalty_events = []
    period = None

    # Loop through table rows
    for row in penalties_table.find_all('tr'):
        if 'stats-section' in row.get('class', []):
            td = row.find('td')
            if td:
                period = td.text.strip()
            else:
                logging.warning("Period name not found in 'stats-section' row")
                period = "Unknown"
        else:
            cols = row.find_all('td')
            if len(cols) > 1:
                team = cols[0].text.strip()
                player = cols[1].text.strip()
                pen_length = cols[2].text.strip()
                penalty_type = cols[3].text.strip()
                time = cols[4].text.strip()

                penalty_event = {
                    'Period': period,
                    'Team': team,
                    'Player': player,
                    'Pen_Length': pen_length,
                    'Penalty_Type': penalty_type,
                    'Time': time
                }
                penalty_events.append(penalty_event)

    return pd.DataFrame(penalty_events)


############# PARSEING GOALIE SUMMARY WITH BS4

def parse_goalie_stats(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the goalies div and table
    goalies_div = soup.find('div', id='goalies')
    if goalies_div is None:
        logging.error("Goalies div not found")
        return None

    goalies_table = goalies_div.find('table')
    if goalies_table is None:
        logging.error("Goalies table not found within the goalies div")
        return None

    # Initialize list to store goalie stats
    goalie_stats = []
    team = None

    # Loop through table rows
    for row in goalies_table.find_all('tr'):
        if 'stats-header' in row.get('class', []):
            td = row.find('td')
            if td:
                team = td.text.strip()
            else:
                logging.warning("Team name not found in 'stats-header' row")
                team = "Unknown"
        else:
            cols = row.find_all('td')
            if len(cols) > 1:
                goalie = cols[0].text.strip()
                sv = cols[1].text.strip()
                ga = cols[2].text.strip()
                minutes = cols[3].text.strip()

                goalie_stat = {
                    'Team': team,
                    'Goalie': goalie,
                    'SV': sv,
                    'GA': ga,
                    'Minutes': minutes
                }
                goalie_stats.append(goalie_stat)

    return pd.DataFrame(goalie_stats)


#### PARSE THE ADVANCED TEAM METRICS TABLES ####
### NEW - IT WORKS IN THE NOTEBOOK BUT NOT IN THE FUNCTION
### RETURNS WHOLE ADVANCED METRICS AS SINGLE TABLE
####################################
def parse_new_advanced_metrics(html_content):
    # Parse HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all tables with advanced metrics
    tables = soup.find_all('table', {'class': 'sortable metrics'})
    
    # List to store all parsed data
    all_data = []
    
    for table in tables:
        # Extract team name from the table header
        team_name = table.find('td').text.strip()
        
        # Extract headers (skipping the Player header)
        headers = [header.text for header in table.find_all('th')][1:]
        
        # Prepare final column headers
        col_names = ['Team', 'Player']
        for header in headers:
            col_names.append(header)
        
        # Extract player data
        rows = table.find_all('tr')[2:]  # skipping the two header rows
        for row in rows:
            player_data = [team_name]  # start with team name
            cells = row.find_all('td')
            player_data.append(cells[0].text.strip())  # player name
            for cell in cells[1:]:
                player_data.append(cell.text.strip())
            all_data.append(player_data)
    
    # Convert the list of data to a DataFrame
    df = pd.DataFrame(all_data, columns=col_names)
    return df

# Testing the new function with the provided HTML content
# advanced_df = parse_new_advanced_metrics(advanced_tie_game_html_content)
# advanced_df


######## NEWS TEST ###############  
def parse_advanced_metrics_tables(html_content):
    # Parse HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all tables with advanced metrics
    tables = soup.find_all('table', {'class': 'sortable metrics'})
    
    # List to store all parsed data
    all_data = []
    
    for table in tables:
        # Extract team name from the table header
        team_name = table.find('td').text.strip()
        
        # Extract headers (skipping the Player header)
        headers = [header.text for header in table.find_all('th')][1:]
        
        # Prepare final column headers
        col_names = ['Team', 'Player']
        for header in headers:
            col_names.append(header)
        
        # Extract player data
        rows = table.find_all('tr')[2:]  # skipping the two header rows
        for row in rows:
            player_data = [team_name]  # start with team name
            cells = row.find_all('td')
            player_data.append(cells[0].text.strip())  # player name
            for cell in cells[1:]:
                player_data.append(cell.text.strip())
            all_data.append(player_data)
    
    # Convert the list of data to a DataFrame
    df = pd.DataFrame(all_data, columns=col_names)
    return df


######### OLDER FUNCTION for Test ################
# ########### UPDATED FUNCTION TO STORE TEAM NAMES IN THE DATAFRAMES
# def parse_advanced_metrics_tables(html_content):
#     # Initialize list to store DataFrames
#     dfs = []
    
#     # Parse HTML content
#     soup = BeautifulSoup(html_content, 'html.parser')
    
#     # Find all tables
#     tables = soup.find_all('table', {'class': 'sortable metrics'})
    
#     for table in tables:
#         # Extract team name
#         team_name = table.find('td').text
        
#         # Initialize list to store column names and data
#         col_names = []
#         col_names_final = []
#         data = []
        
#         # Get headers
#         headers = table.find_all('th')
#         for header in headers:
#             col_names.append(header.text)
        
#         # Add TOTAL, EVEN STRENGTH, POWER PLAY, CLOSE to column names
#         section_headers = ['TOTAL', 'EVEN STRENGTH', 'POWER PLAY', 'CLOSE']
#         for col in col_names:
#             for section in section_headers:
#                 if col in section_headers:
#                     temp_col = section
#                 else:
#                     temp_col = f"{section}_{col}"
#             col_names_final.append(temp_col)
        
#         # Get data rows
#         rows = table.find_all('tr')[2:]  # skip header rows
#         for row in rows:
#             row_data = [team_name]  # Add team name as the first element
#             cells = row.find_all('td')
#             for cell in cells:
#                 row_data.append(cell.text.strip())
#             data.append(row_data)
        
#         # Add "Team" to the column names
#         new_names = ['Team', 'Player', 'TOTAL_Block', 'TOTAL_Miss', 'TOTAL_Saved', 'TOTAL_Goals', 'TOTAL_Total_Shots',
#                      'EVEN_Block', 'EVEN_Miss', 'EVEN_Saved', 'EVEN_Goals', 'EVEN_Total_Shots',
#                      'PP_Block', 'PP_Miss', 'PP_Saved', 'PP_Goals', 'PP_Total_Shots',
#                      'CLOSE_Block', 'CLOSE_Miss', 'CLOSE_Saved', 'CLOSE_Goals', 'CLOSE_Total_Shots',
#                      'D_Blocks', 'Faceoffs']

#         # Create DataFrame and set the column names
#         df = pd.DataFrame(data, columns=new_names)

#         # Append DataFrame to list
#         dfs.append(df)
    
#     return dfs


# Complete code for parsing the line chart information with specific positions for forwards and defensemen.


def parse_line_chart(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    line_chart_div = soup.find('div', id='linechart')

    if line_chart_div is None:
        logging.error("Line chart div not found")
        return pd.DataFrame()

    line_data = []

    for team_div in line_chart_div.find_all('div', recursive=False):
        h3 = team_div.find('h3')
        if h3 is None:
            logging.warning("Team name not found")
            continue
        
        team_name = h3.text.strip()
        
        for line_type_div in team_div.find_all('div', recursive=False):
            line_type = line_type_div.get('class')[0] if line_type_div.get('class') else None
            if line_type is None:
                logging.warning("Line type not found")
                continue
            
            if line_type == 'f':
                position_types = ['Left Wing', 'Center', 'Right Wing']
            elif line_type == 'd':
                position_types = ['Left D', 'Right D']
            elif line_type == 'x':
                position_types = ['Extra']
            elif line_type == 'g':
                position_types = ['Goalie']
                goalie_count = 1  # Initialize goalie count
            else:
                continue

            players = line_type_div.find_all('div')
            if not players:
                logging.warning(f"No players found for {team_name} in {line_type}")
                continue
            
            for i, player in enumerate(players):
                player_name = player.text.strip()
                if line_type == 'x':
                    player_name = player_name.split(' ')[0]
                if line_type == 'g':
                    line_number = f"Goalie {goalie_count}"
                    goalie_count += 1
                else:
                    line_number = i // len(position_types) + 1

                position = position_types[i % len(position_types)]
                line_data.append({
                    'Team': team_name,
                    'Line': line_number,
                    'Position': position,
                    'Player': player_name
                })

    if not line_data:
        logging.error("No line data was collected")

    df = pd.DataFrame(line_data)
    
    # # Log DataFrame info for debugging
    # if df.empty:
    #     logging.warning("Generated line chart DataFrame is empty.")
    # else:
    #     logging.info(f"Generated line chart DataFrame with columns: {df.columns.tolist()}")

    return df

### Get the Linescore Elements - Score, shots, ect by period####
### UPDATED 

def parse_linescore(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    linescore_data = []
    
    # Parsing the Goals table
    goals_table = soup.select_one("#goals table")
    if goals_table is None:
        logging.error("Goals table not found")
        return None
    
    rows = goals_table.select('tbody tr')
    if not rows:
        logging.warning("No rows found in Goals table")
        return None
    
    for row in rows:
        team_data = {}
        td = row.select_one('td')
        if td:
            team_data['Team'] = td.text
        else:
            logging.warning("Team name not found in Goals table")
            continue

        goals = row.select('td')[1:]
        for i, goal in enumerate(goals):
            period = i + 1
            column_name = f'goals{period}' if i < len(goals) - 1 else 'goalsT'
            team_data[column_name] = int(goal.text)
        
        linescore_data.append(team_data)
    

    # Parsing the Shots table
    shots_table = soup.select_one("#shots table")
    if shots_table is None:
        logging.error("Shots table not found")
        return None

    rows = shots_table.select('tbody tr')
    if not rows:
        logging.warning("No rows found in Shots table")
        return None

    for i, row in enumerate(rows):
        shots = row.select('td')[1:]
        if not shots:
            logging.warning(f"No shot data found for row {i+1} in Shots table")
            continue

        for j, shot in enumerate(shots):
            period = j + 1
            column_name = f'shots{period}' if j < len(shots) - 1 else 'shotsT'
            try:
                linescore_data[i][column_name] = int(shot.text.strip())
            except ValueError:
                logging.warning(f"Could not convert shot data to integer for row {i+1}, column {j+1}")
                linescore_data[i][column_name] = None

    # Parsing the PP table
    pp_table = soup.select_one("#pp table")
    if pp_table is None:
        logging.error("PP table not found")
        return None

    rows = pp_table.select('tbody tr')
    if not rows:
        logging.warning("No rows found in PP table")
        return None

    for i, row in enumerate(rows):
        try:
            pen_pim = row.select('td')[1].text.split('‑')
            linescore_data[i]['Pen'] = int(pen_pim[0])
            linescore_data[i]['PIM'] = int(pen_pim[1])

            ppg_ppo = row.select('td')[2].text.split('‑')
            linescore_data[i]['PPG'] = int(ppg_ppo[0])
            linescore_data[i]['PPO'] = int(ppg_ppo[1])

            fow_fol = row.select('td')[3].text.split('‑')
            linescore_data[i]['FOW'] = int(fow_fol[0])
            linescore_data[i]['FOL'] = int(fow_fol[1])
            linescore_data[i]['FOW%'] = (linescore_data[i]['FOW'] / (linescore_data[i]['FOW'] + linescore_data[i]['FOL'])) * 100

        except (ValueError, IndexError) as e:
            logging.warning(f"Could not process PP data for row {i+1}. Error: {e}")
            continue

    # Convert to DataFrame early
    df = pd.DataFrame(linescore_data)

    # Ensure all columns exist
    expected_goals_columns = [f'goals{i}' for i in range(1, 5)] + ['goalsT']
    expected_shots_columns = [f'shots{i}' for i in range(1, 5)] + ['shotsT']

    for col in expected_goals_columns + expected_shots_columns:
        if col not in df.columns:
            df[col] = 0

    # Return the final DataFrame
    return df


# ############################## ORIGINAL FUNCTION #########################################
# def parse_linescore(html_content):
#     soup = BeautifulSoup(html_content, 'html.parser')
#     linescore_data = []
    
#     # Parsing the Goals table
#     goals_table = soup.select_one("#goals table")
#     if goals_table is None:
#         logging.error("Goals table not found")
#         return None
    
#     rows = goals_table.select('tbody tr')
#     if not rows:
#         logging.warning("No rows found in Goals table")
#         return None
    
#     for row in rows:
#         team_data = {}
#         td = row.select_one('td')
#         if td:
#             team_data['Team'] = td.text
#         else:
#             logging.warning("Team name not found in Goals table")
#             continue

#         goals = row.select('td')[1:]
#         for i, goal in enumerate(goals):
#             period = i + 1
#             column_name = f'goals{period}' if i < len(goals) - 1 else 'goalsT'
#             team_data[column_name] = int(goal.text)
        
#         linescore_data.append(team_data)
    

#     # Parsing the Shots table
#     shots_table = soup.select_one("#shots table")
#     if shots_table is None:
#         logging.error("Shots table not found")
#         return None

#     rows = shots_table.select('tbody tr')
#     if not rows:
#         logging.warning("No rows found in Shots table")
#         return None

#     for i, row in enumerate(rows):
#         shots = row.select('td')[1:]
#         if not shots:
#             logging.warning(f"No shot data found for row {i+1} in Shots table")
#             continue

#         for j, shot in enumerate(shots):
#             period = j + 1
#             column_name = f'shots{period}' if j < len(shots) - 1 else 'shotsT'
#             try:
#                 linescore_data[i][column_name] = int(shot.text.strip())
#             except ValueError:
#                 logging.warning(f"Could not convert shot data to integer for row {i+1}, column {j+1}")
#                 linescore_data[i][column_name] = None

#     # Parsing the PP table
#     pp_table = soup.select_one("#pp table")
#     if pp_table is None:
#         logging.error("PP table not found")
#         return None

#     rows = pp_table.select('tbody tr')
#     if not rows:
#         logging.warning("No rows found in PP table")
#         return None

#     for i, row in enumerate(rows):
#         try:
#             pen_pim = row.select('td')[1].text.split('‑')
#             linescore_data[i]['Pen'] = int(pen_pim[0])
#             linescore_data[i]['PIM'] = int(pen_pim[1])

#             ppg_ppo = row.select('td')[2].text.split('‑')
#             linescore_data[i]['PPG'] = int(ppg_ppo[0])
#             linescore_data[i]['PPO'] = int(ppg_ppo[1])

#             fow_fol = row.select('td')[3].text.split('‑')
#             linescore_data[i]['FOW'] = int(fow_fol[0])
#             linescore_data[i]['FOL'] = int(fow_fol[1])
#             linescore_data[i]['FOW%'] = (linescore_data[i]['FOW'] / (linescore_data[i]['FOW'] + linescore_data[i]['FOL'])) * 100

#         except (ValueError, IndexError) as e:
#             logging.warning(f"Could not process PP data for row {i+1}. Error: {e}")
#             continue

#     return pd.DataFrame(linescore_data)



# Function to parse game details


def parse_game_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    meta_div = soup.find('div', {'id': 'meta'})
    if meta_div is None:
        logging.error("Meta div not found")
        return None
    
    game_details_div = meta_div.find_all('div')[-1]
    if game_details_div is None:
        logging.error("Game details div not found")
        return None
    
    try:
        date_str = game_details_div.h4.string
        day_of_week, date = date_str.split(", ", 1)
        
        p_elements = game_details_div.find_all('p')
        
        # Extract conference and location details
        for p in p_elements:
            if "Game" in p.text:  # e.g., "Big Ten Game"
                details_strs = p.get_text(separator='|').split('|')
                conference = details_strs[0]
                location = details_strs[-1].split('at ')[-1]
                break
        else:  # Defaults if not found
            conference, location = None, None
        
        # Extract referees and assistant referees details
        for p in p_elements:
            if "Referees" in p.text:
                refs_str = p.strong.next_sibling if p.strong else None
                asst_refs_str = p.find_all('strong')[1].next_sibling if len(p.find_all('strong')) > 1 else None
                break
        else:  # Defaults if not found
            refs_str, asst_refs_str = None, None
        
        refs = refs_str.split(', ') if refs_str else []
        asst_refs = asst_refs_str.split(', ') if asst_refs_str else []
        refs = [re.sub(r'[^a-zA-Z ]+', '', ref).strip() for ref in refs]
        asst_refs = [re.sub(r'[^a-zA-Z ]+', '', ref).strip() for ref in asst_refs]
        
        # Extract attendance details using regex for better accuracy
        attendance_pattern = r"Attendance:\s?(\d+[\d,]*)"
        attendance_match = re.search(attendance_pattern, html_content)
        attendance = int(attendance_match.group(1).replace(',', '')) if attendance_match else None
        
        # Extract game details (like shootout results)
        details = None
        for p in p_elements:
            if "shootout" in p.text:
                details = p.text
                break
        
        # Clean details if present
        if details and '\n' in details:
            details = details.replace('\n', '').strip()
        if details and '\t' in details:
            details = re.sub('\t', ' ', details)
        
        game_details = {
            'Day': day_of_week,
            'Date': date,
            'Conference': conference,
            'Details': details,
            'Location': location,
            'Ref1': refs[0] if refs else None,
            'Ref2': refs[1] if len(refs) > 1 else None,
            'Asst_Ref1': asst_refs[0] if asst_refs else None,
            'Asst_Ref2': asst_refs[1] if len(asst_refs) > 1 else None,
            'Attendance': attendance
        }
        
        game_details_df = pd.DataFrame([game_details])
        return game_details_df

    except (AttributeError, IndexError, ValueError) as e:
        logging.error(f"Error while parsing game details: {e}")
        return None


def parse_box_score(box_score_html):
    # Initialize DataFrames to None
    scoring_summary = penalty_summary = goalie_stats = player_stats = line_chart = linescore = game_details = None
    
    try:
        scoring_summary = parse_scoring_summary(box_score_html)
    except Exception as e:
        print(f"Error in parse_scoring_summary: {e}")
    
    try:
        penalty_summary = parse_penalty_summary(box_score_html)
    except Exception as e:
        print(f"Error in parse_penalty_summary: {e}")
    
    try:
        goalie_stats = parse_goalie_stats(box_score_html)
    except Exception as e:
        print(f"Error in parse_goalie_stats: {e}")
    
    try:
        player_stats = parse_player_summary(box_score_html)
    except Exception as e:
        print(f"Error in parse_player_summary: {e}")
    
    try:
        line_chart = parse_line_chart(box_score_html)
        if line_chart.empty:
            logging.info("Line chart is empty. Skipping the insert for this game.")
        else:
            logging.info(f"Line chart DataFrame structure: {line_chart.dtypes}")

        # Insert into database (make sure this part works as expected)

    except Exception as e:
        logging.error(f"Error in parse_line_chart: {e}")


    
    try:
        linescore = parse_linescore(box_score_html)
    except Exception as e:
        print(f"Error in parse_linescore: {e}")
    
    try:
        game_details = parse_game_details(box_score_html)
    except Exception as e:
        print(f"Error in parse_game_details: {e}")
    
    # Combine DataFrames into a list
    all_dfs = [game_details, scoring_summary, penalty_summary, goalie_stats, player_stats, line_chart, linescore]
    
    return all_dfs


def rename_duplicate_columns(df):
    cols = pd.Series(df.columns)
    for dup in df.columns[df.columns.duplicated()].unique(): 
        cols[df.columns.get_loc(dup)] = [f"{dup}_{i}" if i != 0 else dup for i in range(df.columns.get_loc(dup).sum())]
    df.columns = cols
    return df




# Function to save DataFrames to SQLite database
import sqlite3
from sqlalchemy import create_engine

# Modified Function to save DataFrames to SQLite database
def save_to_sqlite_db(df_list, table_names, tag=''):
    db_name = f"../TEMP/{FILE_TAG}_Game_Stats.db"
    engine = create_engine(f'sqlite:///{db_name}')
    
    for df, table in zip(df_list, table_names):
        try:
            df = rename_duplicate_columns(df)
            df.to_sql(table, engine, if_exists='append', index=False)
        except Exception as e:
            print(f"Error saving to table {table}: {e}")

############# OLDER FUNCTION ################
# def save_to_sqlite_db(df_list, table_names, tag=''):
#     db_name = f"../TEMP/NEWEST_Scrape_Game_Stats.db"  # Create DB file name using the TAG
#     engine = create_engine(f'sqlite:///{db_name}')
    
#     for df, table in zip(df_list, table_names):
#         # Assuming rename_duplicate_columns is a function you've defined earlier
#         df = rename_duplicate_columns(df)
#         df.to_sql(table, engine, if_exists='append', index=False)

# Function to fetch and save data
# Modified Function to fetch and save data
def fetch_and_save_data_to_db(box_score_url, advanced_metrics_url, tag=f'{tag}'):  # Added tag parameter
    db_name = f"{tag} CHN Data.db"  # Create DB name using the TAG
    # Fetch HTML content for box score
    box_score_response = requests.get(box_score_url)
    box_score_html = box_score_response.text
    
    # Fetch HTML content for advanced metrics
    advanced_metrics_response = requests.get(advanced_metrics_url)
    advanced_metrics_html = advanced_metrics_response.text
    
    # Parse box score into list of DataFrames
    box_score_dfs = parse_box_score(box_score_html)
    
    # Parse advanced metrics into list of DataFrames
    advanced_metrics_df = parse_advanced_metrics_tables(advanced_metrics_html)
    advanced_metrics_dfs = [advanced_metrics_df]
    
    # Combine all DataFrames into a list
    all_dfs = box_score_dfs + advanced_metrics_dfs
    
    # Define table names for these DataFrames
    table_names = ['game_details', 'scoring_summary', 'penalty_summary', 
                    'goalie_stats', 'player_stats', 'line_chart', 'linescore',
                    'advanced_metrics']


    # if len(all_dfs) != len(table_names):
    #     raise ValueError("Mismatch between number of DataFrames and table names!")

    # Diagnostic Step 1: Print the number of DataFrames and table names
    # print(f"Number of DataFrames: {len(all_dfs)}")
    # print(f"Number of table names: {len(table_names)}")

    # Diagnostic Step 2: Print the names of the columns for each DataFrame
    # for df in all_dfs:
    #     print(df.columns.tolist())


    # Create a game_id for the game and apply it to all dataframes
    # Game ID YYYMMDD-HomeTeam-AwayTeam
    for df in all_dfs:
        df['Game_ID'] = game_id

        # Diagnostic check
    if len(all_dfs) != len(table_names):
        print(f"Mismatch detected!")
        print(f"box_score_url: {box_score_url}")
        print(f"advanced_metrics_url: {advanced_metrics_url}")
    
    # Save DataFrames to SQLite database
    save_to_sqlite_db(all_dfs, table_names, tag)  # Pass the tag here
    
    return all_dfs


In [5]:
# # Example usage:
# TAG = "NEW2022-2023"
# save_to_sqlite_db(df_list, table_names, TAG)
# fetch_and_save_data_to_db(box_score_url, advanced_metrics_url, TAG)

In [6]:
## Run the scrape to get game data using the functions above and infor from games_df

## Change the variable name to reuse the old code
sampled_games = games_df

# Initialize counters & logs
error_count = 0
error_games = []

# Loop over sampled games and fetch data
for idx, row in tqdm(sampled_games.iterrows(), total=sampled_games.shape[0], desc="Scraping games"):
    retries = 3  # Number of retries
    success = False

    
    while retries > 0 and not success:
        try:
            box_score_url = base_url + row['Box_Link']
            advanced_metrics_url = base_url + row['Metrics_Link']

            # create a unique game id
            game_id = str(row['Date']) + '-' + str(row['Home_Team']) + '-' + str(row['Away_Team'])
            
            logging.info(f"Fetching data for game: {row['Home_Team']} vs {row['Away_Team']}")
            
            # Your existing function to fetch and save data
            all_dfs = fetch_and_save_data_to_db(box_score_url, advanced_metrics_url)
            
            # If reached here, the fetching was successful
            success = True
            
            # Adaptive rate limiting
            
            
        except requests.exceptions.RequestException as e:  # Network-related errors
            logging.error(f"Network error for game: {row['Home_Team']} vs {row['Away_Team']}. Error: {e}")
            retries -= 1
            time.sleep(5)  # Wait for 5 seconds before retrying
        
        except Exception as e:  # Other exceptions
            logging.error(f"An error occurred for game: {row['Game_ID']} - {row['Home_Team']} vs {row['Away_Team']}. Error: {e}")
            error_count += 1
            error_games.append((row['Home_Team'], row['Away_Team']))
            break  # Break the while loop; no retries for these types of errors


# Close the logging file
logging.shutdown()

# Close the database connection
# conn.close()



Scraping games:   0%|          | 0/1106 [00:00<?, ?it/s]

Scraping games:   2%|▏         | 26/1106 [01:10<52:24,  2.91s/it]

Error saving to table line_chart: (sqlite3.OperationalError) table line_chart has no column named Team
[SQL: INSERT INTO line_chart ("Team", "Line", "Position", "Player", "Game_ID") VALUES (?, ?, ?, ?, ?)]
[parameters: [('Boston College', 1, 'Left Wing', 'Brandon Kruse', '2021-10-08-Boston College-Quinnipiac'), ('Boston College', 1, 'Center', 'Jack McBain', '2021-10-08-Boston College-Quinnipiac'), ('Boston College', 1, 'Right Wing', 'Colby Ambrosio', '2021-10-08-Boston College-Quinnipiac'), ('Boston College', 2, 'Left Wing', 'Trevor Kuntar', '2021-10-08-Boston College-Quinnipiac'), ('Boston College', 2, 'Center', 'Marc McLaughlin', '2021-10-08-Boston College-Quinnipiac'), ('Boston College', 2, 'Right Wing', 'Nikita Nesterenko', '2021-10-08-Boston College-Quinnipiac'), ('Boston College', 3, 'Left Wing', 'Mike Posma', '2021-10-08-Boston College-Quinnipiac'), ('Boston College', 3, 'Center', 'Patrick Giles', '2021-10-08-Boston College-Quinnipiac')  ... displaying 10 of 42 total bound param

Scraping games:   2%|▏         | 27/1106 [01:13<51:48,  2.88s/it]

Error saving to table line_chart: (sqlite3.OperationalError) table line_chart has no column named Team
[SQL: INSERT INTO line_chart ("Team", "Line", "Position", "Player", "Game_ID") VALUES (?, ?, ?, ?, ?)]
[parameters: [('Holy Cross', 1, 'Left Wing', 'Jack Ricketts', '2021-10-08-Holy Cross-Northeastern'), ('Holy Cross', 1, 'Center', 'Matt Guerra', '2021-10-08-Holy Cross-Northeastern'), ('Holy Cross', 1, 'Right Wing', 'Alex Peterson', '2021-10-08-Holy Cross-Northeastern'), ('Holy Cross', 2, 'Left Wing', 'Tyler Ghirardosi', '2021-10-08-Holy Cross-Northeastern'), ('Holy Cross', 2, 'Center', 'Ryan Leibold', '2021-10-08-Holy Cross-Northeastern'), ('Holy Cross', 2, 'Right Wing', 'Anthony Vincent', '2021-10-08-Holy Cross-Northeastern'), ('Holy Cross', 3, 'Left Wing', 'Lucas Thorne', '2021-10-08-Holy Cross-Northeastern'), ('Holy Cross', 3, 'Center', 'Daniel Colabufo', '2021-10-08-Holy Cross-Northeastern')  ... displaying 10 of 44 total bound parameter sets ...  ('Northeastern', 'Goalie 2', 'Go

...

Scraping games:  42%|████▏     | 468/1106 [22:17<30:19,  2.85s/it]

Error saving to table line_chart: (sqlite3.OperationalError) table line_chart has no column named Team
[SQL: INSERT INTO line_chart ("Team", "Line", "Position", "Player", "Game_ID") VALUES (?, ?, ?, ?, ?)]
[parameters: [('Maine', 1, 'Left Wing', 'Ben Poisson', '2021-12-04-Vermont-Maine'), ('Maine', 1, 'Center', 'Lynden Breen', '2021-12-04-Vermont-Maine'), ('Maine', 1, 'Right Wing', 'Donavan Villeneuve-Houle', '2021-12-04-Vermont-Maine'), ('Maine', 2, 'Left Wing', 'Keenan Suthers', '2021-12-04-Vermont-Maine'), ('Maine', 2, 'Center', 'Nolan Renwick', '2021-12-04-Vermont-Maine'), ('Maine', 2, 'Right Wing', 'John Mulera', '2021-12-04-Vermont-Maine'), ('Maine', 3, 'Left Wing', 'AJ Drobot', '2021-12-04-Vermont-Maine'), ('Maine', 3, 'Center', 'Jack Quinlivan', '2021-12-04-Vermont-Maine')  ... displaying 10 of 40 total bound parameter sets ...  ('Vermont', 1, 'Extra', 'Cody', '2021-12-04-Vermont-Maine'), ('Vermont', 'Goalie 1', 'Goalie', 'Gabe Carriere', '2021-12-04-Vermont-Maine')]]
(Backgrou

Scraping games:  42%|████▏     | 469/1106 [22:20<30:40,  2.89s/it]

Error saving to table line_chart: (sqlite3.OperationalError) table line_chart has no column named Team
[SQL: INSERT INTO line_chart ("Team", "Line", "Position", "Player", "Game_ID") VALUES (?, ?, ?, ?, ?)]
[parameters: [('Mass.-Lowell', 1, 'Left Wing', 'Andre Lee', '2021-12-04-Mass.-Lowell-Massachusetts'), ('Mass.-Lowell', 1, 'Center', 'Carl Berglund', '2021-12-04-Mass.-Lowell-Massachusetts'), ('Mass.-Lowell', 1, 'Right Wing', 'Brian Chambers', '2021-12-04-Mass.-Lowell-Massachusetts'), ('Mass.-Lowell', 2, 'Left Wing', 'Blake Wells', '2021-12-04-Mass.-Lowell-Massachusetts'), ('Mass.-Lowell', 2, 'Center', 'Connor Sodergren', '2021-12-04-Mass.-Lowell-Massachusetts'), ('Mass.-Lowell', 2, 'Right Wing', 'Matt Allen', '2021-12-04-Mass.-Lowell-Massachusetts'), ('Mass.-Lowell', 3, 'Left Wing', 'Reid Stefanson', '2021-12-04-Mass.-Lowell-Massachusetts'), ('Mass.-Lowell', 3, 'Center', 'Lucas Condotta', '2021-12-04-Mass.-Lowell-Massachusetts')  ... displaying 10 of 44 total bound parameter sets ...

Scraping games:  48%|████▊     | 528/1106 [25:09<27:08,  2.82s/it]

In [ ]:
## Print a summary of the errors
print(f"Total games: {sampled_games.shape[0]}")
print(f"Games With Errors: {error_count}")
for game in error_games:
    print(f"Error games: {game}")


Total games: 1106
Games With Errors: 25
Error games: ("American Int'l", 'Boston College')
Error games: ('Rensselaer', 'Union')
Error games: ('St. Lawrence', 'Clarkson')
Error games: ('Bowling Green', 'Michigan')
Error games: ('Bemidji State', 'North Dakota')
Error games: ('Holy Cross', 'Boston University')
Error games: ('Mercyhurst', 'Vermont')
Error games: ('Air Force', 'Colorado College')
Error games: ('US Under-18', 'Northern Michigan')
Error games: ('St. Lawrence', 'RIT')
Error games: ('US Under-18', 'Michigan Tech')
Error games: ('Assumption', 'Long Island')
Error games: ('Post', 'Long Island')
Error games: ('Air Force', 'Lindenwood')
Error games: ('Dartmouth', 'Massachusetts')
Error games: ('US Under-18', 'Cornell')
Error games: ('Air Force', 'Lindenwood')
Error games: ('US Under-18', 'Mercyhurst')
Error games: ('Sacred Heart', 'Yale')
Error games: ('Long Island', 'St. Anselm')
Error games: ('US Under-18', 'Boston University')
Error games: ('US Under-18', 'Merrimack')
Error games

## 11-2 Scrape is working well
- wanto to get better werror reporting for each fuynction. list of what failed

# The Errors are coming from OT Games!

## Fix / Notes



# DB Cleaning Below Next

In [ ]:
enda;sodfgjas;d

NameError: name 'enda' is not defined

### Try to visualize advanced metrics here in the notebook

In [ ]:
# combined_sample_data.head(10)

# combined_sample_data.info()